<a href="https://colab.research.google.com/github/gagan3012/keytotext/blob/master/notebooks/Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keytotext --upgrade
!sudo apt-get install git-lfs

     |████████████████████████████████| 819kB 8.9MB/s 
     |████████████████████████████████| 266kB 54.2MB/s 
     |████████████████████████████████| 1.2MB 38.1MB/s 
     |████████████████████████████████| 1.8MB 42.0MB/s 
     |████████████████████████████████| 2.5MB 30.9MB/s 
     |████████████████████████████████| 235kB 54.3MB/s 
     |████████████████████████████████| 645kB 47.4MB/s 
     |████████████████████████████████| 829kB 56.2MB/s 
     |████████████████████████████████| 122kB 51.9MB/s 
     |████████████████████████████████| 10.6MB 45.7MB/s 
     |████████████████████████████████| 245kB 41.2MB/s 
     |████████████████████████████████| 143kB 56.0MB/s 
     |████████████████████████████████| 174kB 53.9MB/s 
     |████████████████████████████████| 102kB 15.3MB/s 
     |████████████████████████████████| 901kB 44.6MB/s 
     |████████████████████████████████| 3.3MB 41.3MB/s 
     |████████████████████████████████| 1.3MB 40.6MB/s 
     |████████████████████████████████| 71kB 11.

In [ ]:
from keytotext import trainer, make_dataset

train_df = make_dataset('common_gen', split='train')
test_df = make_dataset('common_gen', split='test')

model = trainer()
model.from_pretrained(model_name="t5-small")
model.train(train_df=train_df[:100], test_df=test_df[:50], batch_size=2, max_epochs=3,use_gpu=True)
model.save_model()

Global seed set to 42


Using custom data configuration default


Dataset common_gen downloaded and prepared to /root/.cache/huggingface/datasets/common_gen/default/2020.5.30/1a9e8bdc026c41ce7a9e96260debf7d2809cb7fd63fa02b017e4fac1b00c6b23. Subsequent calls will reuse this data.


Using custom data configuration default
Reusing dataset common_gen (/root/.cache/huggingface/datasets/common_gen/default/2020.5.30/1a9e8bdc026c41ce7a9e96260debf7d2809cb7fd63fa02b017e4fac1b00c6b23)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc



  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Global seed set to 42


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: LightningDeprecationWarning: Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2 and will be removed in v1.4. Please, create your own `mc = ModelCheckpoint(monitor='your_monitor')` and use it as `Trainer(callbacks=[mc])`.
  "Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2"



Enter your HuggingFace password··········
Check out your model at:
https://huggingface.co/gagan3012/k2t-test3


In [ ]:
model.upload(hf_username="gagan3012",model_name="k2t-test3")

In [ ]:
model.load_model("gagan3012/k2t-test3", use_gpu=True)

keywords=["ski", "mountain", "sky"]
model.predict(keywords)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


'ski mountain sky'